In [ ]:
import numpy as np
import pandas as pd
import os

### CIFAR10 Dataset Preprocessing and Split

In [ ]:
from tensorflow.keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = \
    cifar10.load_data()

print('train dataset shape: ', train_images.shape, train_labels.shape)
print('test dataset shape: ', test_images.shape, test_labels.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10

def get_preprocessed_ohe(images, labels, pre_func=None):
    if pre_func is not None:
        image = pre_func(images)
    oh_labels = to_categorical(labels)
    return images, oh_labels

def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels)


In [ ]:
(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)

print('train dataset shape: ', tr_images.shape, tr_oh_labels.shape)
print('validation dataset shape: ', val_images.shape, val_oh_labels.shape)
print('test dataset shape: ', test_images.shape, test_oh_labels.shape)

### AlexNet Model Create

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras import regularizers

def create_alexnet(in_shape=(227, 227, 3), n_classes=10, kernel_regular=None):
    
    input_tensor = Input(shape=in_shape)
    
    x = Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid')(input_tensor)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=kernel_regular)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = Flatten()(x)
    
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output = Dense(units=n_classes, activation='softmax')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    
    model.summary()

    return model

In [ ]:
model = create_alexnet(in_shape=(227, 227, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))

### CIFAR10 Dataset의 원본 이미지 크기가 32x32이지만, AlexNet 모델이 32x32 크기의 input image를 넣게되면 오류가 발생한다. (64도 마찬가지)

In [ ]:
# model = create_alexnet(in_shape=(32, 32, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))
# model = create_alexnet(in_shape=(64, 64, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))
model = create_alexnet(in_shape=(128, 128, 3), n_classes=10, kernel_regular=regularizers.l2(l2=1e-4))

### CIFAR10 원본 이미지 크기 32x32를 128x128로 증가시키는 Sequence Dataset 생성
- 128x128로 CIFAR10의 모든 이미지 배열 값을 증가시키면 RAM 부족
- 배치 크기 만큼의 개수만 원본 이미지의 크기를 128x128로 증가시킨 뒤, 이를 모델에 입력하는 로직으로 Seqence Dataset을 구성

In [ ]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

In [ ]:
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

class CIFAR_Dataset(Sequence):
    
    def __init__(self, images_array, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        if self.shuffle:
            pass
        
    def __len__(self):
        return int(np.ceil(len(self.labels)/self.batch_size))
    
    def __getitem__(self, index):
        
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        image_batch = np.zeros((images_fetch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        for image_index in range(images_fetch.shape[0]):
            image = cv2.resize(images_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if (self.shuffle):
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass

In [ ]:
def zero_one_scaler(image):
    return image/255.0

In [ ]:
tr_ds = CIFAR_Dataset(
    tr_images,
    tr_oh_labels,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=True,
    pre_func=zero_one_scaler
)

val_ds = CIFAR_Dataset(
    val_images,
    val_oh_labels,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=zero_one_scaler
)

In [ ]:
print('train batch shape: ', next(iter(tr_ds))[0].shape, next(iter(tr_ds))[1].shape)
print('validation batch shape: ', next(iter(val_ds))[0].shape, next(iter(val_ds))[1].shape)


In [ ]:
model = create_alexnet(
    in_shape=(128, 128, 3),
    n_classes=10,
    kernel_regular=regularizers.l2(l2=1e-4)
)

model.compile(
    optimizer=Adam(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    mode='min',
    verbose=1
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    verbose=1
)

In [ ]:
history = model.fit(
    tr_ds,
    epochs=30,
    validation_data=val_ds,
    callbacks=[rlr_cb, ely_cb]
)

In [ ]:
test_ds = CIFAR_Dataset(
    test_images,
    test_oh_labels,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=zero_one_scaler
)

model.evaluate(test_ds)